# Performance Evaluation (Visualization)

This python notebook attempts to recreate all the graphs found in our paper submission. Along with the notebook, we provide a `requirements.txt` file to install all the required python/visualization tools. Simply run `pip install -r requirements.txt` to install them, and then run the following kernels.

In [ ]:
import altair as alt
import numpy as np
import pandas as pd
from vega_datasets import data
from altair_saver import save
import os
import sys

## Path to generated data
Currently points to: `<repo>/docs/data` with all the generated results in csv files.

In [ ]:
data_dir = os.path.abspath("data")

## Figure 2. SpMV runtime comparison versus CUB
Recreates the overhead analysis of SpMV merge-path versus CUB's SpMV implementation.

In [ ]:
versus_cub = os.path.join(data_dir, "versus_cub.csv")
df_overhead = pd.read_csv(versus_cub)

df_overhead["nnzs"] = df_overhead["nnzs"].astype(int)
df_overhead["elapsed"] = df_overhead["elapsed"].astype(float)

In [ ]:
alt.data_transformers.disable_max_rows()

overhead = alt.Chart(df_overhead).mark_point(
    opacity=0.6
).encode(
    x=alt.X(
        "nnzs:Q", 
        scale=alt.Scale(type="log",domain=(1,1000000000)),
#         axis=alt.Axis(tickCount=10),
        title="Number of Nonzeros"
    ),
    y=alt.Y(
        "elapsed:Q", 
        scale=alt.Scale(type="log",domain=(0.001,100)),
        axis=alt.Axis(grid=True), 
        title="Runtime (ms)"
    ),
    color=alt.Color(
        "kernel",
        scale=alt.Scale(range=['#76b900','#DC143C']), # scheme='tableau10'),
        legend=alt.Legend(title="Kernel",orient='top-left',fillColor='#FFFFFF', strokeColor='#E0DCDC', padding=10)
    ),
#     facet=alt.Facet('kernel:O', title=None, columns=2),
    shape=alt.Shape(
        "kernel",
        scale=alt.Scale(range=['cross', 'triangle-right']),
        legend=alt.Legend(title="Kernel",orient='top-left')
    ),
).properties(
    width=700,
    height=500
).configure_point(
    size=6
).interactive()

overhead
# overhead.save(os.path.join(data_dir, "merge_vs_cub.pdf"))

## Figure 3. Complete performance landscape
Recreates the complete performance landscape of our SpMV implementation versus cuSparse.

In [ ]:
cusparse = pd.read_csv(os.path.join(data_dir, "cusparse.csv"))
merge_path = pd.read_csv(os.path.join(data_dir, "merge_path.csv"))
group_mapped = pd.read_csv(os.path.join(data_dir, "group_mapped.csv"))
thread_mapped = pd.read_csv(os.path.join(data_dir, "thread_mapped.csv"))

In [ ]:
alt.data_transformers.disable_max_rows()

landscape = alt.Chart(df_landscape).mark_point(
    opacity=0.7
).encode(
    x=alt.X(
        "nnzs:Q", 
        scale=alt.Scale(type="log",domain=(1,1000000000)),
#         axis=alt.Axis(tickCount=10),
        title="Number of Nonzeros"
    ),
    y=alt.Y(
        "elapsed:Q", 
        scale=alt.Scale(type="log",domain=(0.001,1000)),
        axis=alt.Axis(grid=True), 
        title="Runtime (ms)"
    ),
    color=alt.Color(
        "kernel:O",
        scale=alt.Scale(range=['#76b900','#0047AB','#DC143C','#B983FF']), #scheme='tableau10'),
        legend=alt.Legend(title="Kernel") #,orient='top-left')
    ),
    shape=alt.Shape(
        "kernel:O",
        legend=alt.Legend(title="Kernel",orient='top-left',fillColor='#FFFFFF', strokeColor='#E0DCDC', padding=10)
    ),
    facet=alt.Facet('kernel:N', title=None, spacing={'row':0}, header=alt.Header(labelExpr="''", title=None), columns=2),
).properties(
    width=400,
    height=400
).configure_point(
    size=7
).interactive()

landscape
# landscape.save(os.path.join(data_dir, "performance_landscape.pdf"))

## Figure 4. Heuristics Selection (Speedup)
Shows our heuristics (presented in the paper) result and also the oracle heuristics.

In [ ]:
df_speed_up = pd.read_csv(os.path.join(data_dir, "heuristics.csv"))

In [ ]:
speed_up = alt.Chart(df_speed_up).mark_point(
    opacity=0.7
).encode(
    x=alt.X(
        "nnzs:Q", 
        scale=alt.Scale(type="log",domain=(1,1000000000)),
#         axis=alt.Axis(tickCount=10),
        title="Number of Nonzeros"
    ),
    y=alt.Y(
        "speed-up:Q", 
        scale=alt.Scale(type="log",domain=(0.1,100)),
        axis=alt.Axis(grid=True), 
        title="Speedup w.r.t cuSparse"
    ),
    color=alt.Color(
        "kernel",
        scale=alt.Scale(range=['#0047AB','#DC143C','#B983FF']),
        legend=alt.Legend(title="Kernel",orient='top-left',fillColor='#FFFFFF', strokeColor='#E0DCDC', padding=10)
    ),
    shape=alt.Shape(
        "kernel",
        scale=alt.Scale(range=['square', 'triangle', 'cross']),
        legend=alt.Legend(title="Kernel") #,orient='top-left')
    ),
).properties(
    width=600,
    height=400
)

speed_up_oracle = alt.Chart(df_speed_up).mark_point(
    opacity=0.7
).encode(
    x=alt.X(
        "nnzs:Q", 
        scale=alt.Scale(type="log",domain=(1,1000000000)),
#         axis=alt.Axis(tickCount=10),
        title="Number of Nonzeros"
    ),
    y=alt.Y(
        "oracle-speed-up:Q", 
        scale=alt.Scale(type="log",domain=(0.1,100)),
        axis=alt.Axis(grid=True), 
        title="Oracle Speedup w.r.t cuSparse"
    ),
    color=alt.Color(
        "oracle-speed-up-kernel",
        scale=alt.Scale(range=['#0047AB','#DC143C','#B983FF']),
        legend=alt.Legend(title="Kernel",orient='top-left',fillColor='#FFFFFF', strokeColor='#E0DCDC', padding=10)
    ),
    shape=alt.Shape(
        "oracle-speed-up-kernel",
        scale=alt.Scale(range=['square', 'triangle', 'cross']),
        legend=alt.Legend(title="Kernel") #,orient='top-left')
    ),
).properties(
    width=600,
    height=400
)



# speed_up
speed_up | speed_up_oracle
# speed_up.save(os.path.join(data_dir, "speed_up.pdf"))
# speed_up_oracle.save(os.path.join(data_dir, "speed_up_oracle.pdf"))